In [1]:
%matplotlib notebook
from astropy.table import Table, vstack, join
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
sys.path.append(f'{os.environ["HOME"]}/Projects/planckClusters/catalogs')
from load_catalogs import load_PSZcatalog
from astropy.io.fits import getheader
from utilities import redshifts_from_papers

import warnings
from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)

/home/boada/Projects/swiftXRT/paper/utilities.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
def load_detcats(outpath, fields):
    for field in fields:
        if os.path.exists(f'{outpath}/{field}/{field}_vtp.detect'):
            data_part = Table.read(f'{outpath}/{field}/{field}_vtp.detect', format='fits')
            if len(data_part) > 0:
                data_part['Field'] = field
                try:
                    hdr = getheader(f'{outpath}/{field}/{field}_vtp_high.detect', ext=1)
                except FileNotFoundError:
                    continue
                data_part['Field_EXP'] = hdr['EXPOSURE']
        else:
            continue

        # handle the first pass when we only have one part
        try:
            data = vstack([data, data_part])
        except NameError:
            data = data_part
        
    return data


def load_MCMCfits(outpath, fields):
    for field in fields:
        if os.path.exists(f'{outpath}/{field}/{field}_mcmcfits.txt'):
            try:
                fit_part = Table.read(f'{outpath}/{field}/{field}_mcmcfits.txt', format='ascii', header_start=0)
            except :
                print(field)
                raise (FileNotFoundError)
            src_part = Table.read(f'{outpath}/{field}/{field}_vtp.detect', format='fits')
            
            fit_part.rename_column('ID', 'INDEX')
            try:
                data_part = join(fit_part, src_part, keys='INDEX', join_type='inner')
            except ValueError:
                continue
        else:
            continue

        # handle the first pass when we only have one part
        try:
            data = vstack([data, data_part])
        except NameError:
            data = data_part
        
    return data

def load_spectralfits(outpath, fields):
    for field in fields:
        if os.path.exists(f'{outpath}/{field}/spec_files/{field}_xspec.results'):
            try:
                data_part = Table.read(f'{outpath}/{field}/spec_files/{field}_xspec.results', 
                                       format='ascii', header_start=0)
                if len(data_part) < 1:
                    continue
                
            except ValueError:
                continue
        else:
            continue

        # handle the first pass when we only have one part
        try:
            data = vstack([data, data_part])
        except NameError:
            data = data_part
        
    return data

In [4]:
# get file data
data = load_PSZcatalog()
data = data.sort_values('NAME')
data = redshifts_from_papers(data)

datapath = '../data_full_new'
data.replace(-1, np.nan, inplace=True)

fields = [n.replace(' ', '_') for n in data['NAME']]

fits = load_MCMCfits(datapath, fields)
cats = load_detcats(datapath, fields)
specs = load_spectralfits(datapath, fields)

# check to see which fields have XRT data
field_mask = [True if os.path.isdir(f'{datapath}/{n.replace(" ", "_")}') else False for n in data['NAME']]
                                    
data['SWIFT'] = False
data['SWIFT'] = field_mask

In [5]:
data[['NAME', 'INDEX_PSZ2', 'RA', 'DEC', 'REDSHIFT']][data['SWIFT'] == True][:10]

,NAME,INDEX_PSZ2,RA,DEC,REDSHIFT
42,PSZ1 G018.09+26.07,NaN,253.121758,-0.247058,0.0870
58,PSZ1 G023.53-36.53,NaN,313.528542,-22.841100,0.2840
83,PSZ1 G031.41+28.75,NaN,256.263153,11.460448,0.2300
125,PSZ1 G044.82-31.66,NaN,316.160324,-4.774060,0.2400
130,PSZ1 G045.18-36.47,NaN,320.538497,-6.843424,0.5344
174,PSZ1 G055.72+17.58,NaN,276.342547,27.804647,0.1940
183,PSZ1 G057.42-10.77,NaN,304.356094,16.092529,0.1360
192,PSZ1 G058.77-26.14,NaN,318.089119,8.470705,NaN
198,PSZ1 G059.99+11.06,NaN,285.012283,29.006053,NaN
256,PSZ1 G078.39+46.13,NaN,242.271185,50.209972,0.4360


In [6]:
# add more columns
data['XRT_exp'] = np.nan
data['N_H'] = np.nan

In [7]:
cats.remove_column('R')
df_cats = cats.to_pandas()
df_cats.Field = df_cats.Field.str.replace('_', ' ')

df_specs = specs.to_pandas()
df_specs.FIELD = df_specs.FIELD.str.replace('_', ' ')

In [8]:
df_cats

,INDEX,RA_VTP,RA_ERR_VTP,DEC_VTP,DEC_ERR_VTP,X_VTP,Y_VTP,X_ERR_VTP,Y_ERR_VTP,SRC_AREA,...,X_ERR,Y_ERR,RA,DEC,RA_ERR,DEC_ERR,PSZ_dist,Extended,Field,Field_EXP
0,1,313.521441,0.001330,-22.811574,0.001614,578.288510,603.165132,1.872792,2.463951,5019.795898,...,7.268476,8.338763,313.519246,-22.811193,0.004765,0.005467,1.866628,1,PSZ1 G023.53-36.53,3101.530100
1,1,256.569526,0.000451,11.427246,0.000411,211.410443,463.037718,0.675989,0.627458,174.591660,...,9.993632,8.513018,256.568113,11.426649,0.006551,0.005581,18.048124,-1,PSZ1 G031.41+28.75,230912.300806
2,2,256.568667,0.000225,11.448792,0.000398,212.718445,495.940595,0.336722,0.608334,56.666668,...,4.069845,5.211479,256.568561,11.447537,0.002668,0.003416,17.976263,-1,PSZ1 G031.41+28.75,230912.300806
3,3,256.555757,0.000139,11.425807,0.000131,232.020394,460.826596,0.208752,0.200512,176.441071,...,11.964034,11.716000,256.555715,11.425855,0.007843,0.007680,17.329519,-1,PSZ1 G031.41+28.75,230912.300806
4,4,256.536503,0.000359,11.593522,0.000496,260.983788,716.939687,0.537755,0.756996,310.948486,...,6.278948,6.868659,256.535755,11.592463,0.004116,0.004503,17.876856,-1,PSZ1 G031.41+28.75,230912.300806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,1,348.934928,0.002054,-37.793994,0.001341,513.861871,491.207927,2.478704,2.047580,6934.670898,...,7.635130,6.753793,348.936862,-37.795028,0.005005,0.004427,1.073133,1,PSZ2 G358.98-67.26,1173.416400
864,1,303.146133,0.000335,-41.480888,0.000363,280.219462,522.188015,0.381975,0.555716,156.018997,...,3.434478,4.372390,303.146293,-41.481313,0.002251,0.002866,5.735848,1,PSZ2 G359.07-32.12,17666.435801
865,2,303.111014,0.000702,-41.506233,0.000621,320.470263,483.562520,0.801532,0.950512,425.545502,...,5.230398,5.747870,303.110804,-41.506016,0.003429,0.003768,4.286728,0,PSZ2 G359.07-32.12,17666.435801
866,3,303.093206,0.000322,-41.609985,0.000286,341.093253,325.151179,0.366884,0.437055,150.056763,...,4.052867,4.491315,303.093364,-41.610170,0.002657,0.002944,8.117845,-1,PSZ2 G359.07-32.12,17666.435801


In [9]:
df_specs

,INDEX,SN,CHI2,FLUX,CNT_RATE_NET,CNT_RATE_MODEL,ECF,LUM,FLUX_corr,LUM_corr,CORR,FIELD
0,1,8.5801,0.9225,5.471100e-13,0.023950,0.025276,2.164500e-11,1.243400e+44,7.085648e-12,1.610333e+45,11.951048,PSZ1 G023.53-36.53
1,22,-1.0000,2945.8600,1.706000e-15,0.000052,0.000086,1.995100e-11,2.442100e+41,3.737449e-15,5.350074e+41,1.190768,PSZ1 G031.41+28.75
2,33,5.2303,0.4920,3.999600e-15,0.000151,0.000177,2.256700e-11,5.725300e+41,1.492425e-14,2.136358e+42,2.731435,PSZ1 G031.41+28.75
3,1,15.1945,1.1220,1.243700e-12,0.053760,0.059863,2.077600e-11,1.230500e+44,1.791920e-11,1.772901e+45,13.407976,PSZ1 G055.72+17.58
4,1,15.5937,0.6860,1.688200e-12,0.063250,0.080295,2.102500e-11,7.815400e+43,3.354087e-11,1.552750e+45,18.867831,PSZ1 G057.42-10.77
...,...,...,...,...,...,...,...,...,...,...,...,...
470,3,22.9621,0.9568,7.732100e-13,0.036340,0.036763,2.103200e-11,8.173700e+43,1.042300e-11,1.101829e+45,12.480172,PSZ2 G358.34-47.31
471,4,4.5696,0.5740,4.321400e-14,0.001718,0.001932,2.237200e-11,4.568200e+42,1.302475e-13,1.376861e+43,2.014011,PSZ2 G358.34-47.31
472,1,9.9259,1.5510,1.588700e-12,0.081760,0.076073,2.088400e-11,1.316400e+44,2.696686e-11,2.234479e+45,15.974167,PSZ2 G358.98-67.26
473,1,5.5705,0.9470,3.337700e-14,0.001527,0.001605,2.079200e-11,1.892100e+42,3.921970e-13,2.223315e+43,10.750517,PSZ2 G359.07-32.12


In [10]:
df_data = data[['NAME', 'INDEX_PSZ2', 'RA', 'DEC', 'REDSHIFT', 'XRT_exp', 'N_H']][data['SWIFT']]

In [11]:
df_data

,NAME,INDEX_PSZ2,RA,DEC,REDSHIFT,XRT_exp,N_H
42,PSZ1 G018.09+26.07,NaN,253.121758,-0.247058,0.0870,NaN,NaN
58,PSZ1 G023.53-36.53,NaN,313.528542,-22.841100,0.2840,NaN,NaN
83,PSZ1 G031.41+28.75,NaN,256.263153,11.460448,0.2300,NaN,NaN
125,PSZ1 G044.82-31.66,NaN,316.160324,-4.774060,0.2400,NaN,NaN
130,PSZ1 G045.18-36.47,NaN,320.538497,-6.843424,0.5344,NaN,NaN
...,...,...,...,...,...,...,...
1936,PSZ2 G356.21-43.11,1640.0,317.834006,-44.434423,NaN,NaN,NaN
1938,PSZ2 G356.88-11.33,1643.0,276.510580,-37.097434,NaN,NaN,NaN
1222,PSZ2 G358.34-47.31,1647.0,323.485677,-42.666426,0.2000,NaN,NaN
1223,PSZ2 G358.98-67.26,1650.0,348.926557,-37.779104,0.1786,NaN,NaN


In [12]:
df_m = df_data.merge(df_cats,                                                      
               how='inner',                                              
               left_on='NAME',                                          
               right_on='Field',                                           
               suffixes=('_data', '_detc'))

In [13]:
df_m

,NAME,INDEX_PSZ2,RA_data,DEC_data,REDSHIFT,XRT_exp,N_H,INDEX,RA_VTP,RA_ERR_VTP,...,X_ERR,Y_ERR,RA_detc,DEC_detc,RA_ERR,DEC_ERR,PSZ_dist,Extended,Field,Field_EXP
0,PSZ1 G023.53-36.53,NaN,313.528542,-22.841100,0.2840,NaN,NaN,1,313.521441,0.001330,...,7.268476,8.338763,313.519246,-22.811193,0.004765,0.005467,1.866628,1,PSZ1 G023.53-36.53,3101.530100
1,PSZ1 G031.41+28.75,NaN,256.263153,11.460448,0.2300,NaN,NaN,1,256.569526,0.000451,...,9.993632,8.513018,256.568113,11.426649,0.006551,0.005581,18.048124,-1,PSZ1 G031.41+28.75,230912.300806
2,PSZ1 G031.41+28.75,NaN,256.263153,11.460448,0.2300,NaN,NaN,2,256.568667,0.000225,...,4.069845,5.211479,256.568561,11.447537,0.002668,0.003416,17.976263,-1,PSZ1 G031.41+28.75,230912.300806
3,PSZ1 G031.41+28.75,NaN,256.263153,11.460448,0.2300,NaN,NaN,3,256.555757,0.000139,...,11.964034,11.716000,256.555715,11.425855,0.007843,0.007680,17.329519,-1,PSZ1 G031.41+28.75,230912.300806
4,PSZ1 G031.41+28.75,NaN,256.263153,11.460448,0.2300,NaN,NaN,4,256.536503,0.000359,...,6.278948,6.868659,256.535755,11.592463,0.004116,0.004503,17.876856,-1,PSZ1 G031.41+28.75,230912.300806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,PSZ2 G358.98-67.26,1650.0,348.926557,-37.779104,0.1786,NaN,NaN,1,348.934928,0.002054,...,7.635130,6.753793,348.936862,-37.795028,0.005005,0.004427,1.073133,1,PSZ2 G358.98-67.26,1173.416400
864,PSZ2 G359.07-32.12,1651.0,303.018900,-41.486882,0.1496,NaN,NaN,1,303.146133,0.000335,...,3.434478,4.372390,303.146293,-41.481313,0.002251,0.002866,5.735848,1,PSZ2 G359.07-32.12,17666.435801
865,PSZ2 G359.07-32.12,1651.0,303.018900,-41.486882,0.1496,NaN,NaN,2,303.111014,0.000702,...,5.230398,5.747870,303.110804,-41.506016,0.003429,0.003768,4.286728,0,PSZ2 G359.07-32.12,17666.435801
866,PSZ2 G359.07-32.12,1651.0,303.018900,-41.486882,0.1496,NaN,NaN,3,303.093206,0.000322,...,4.052867,4.491315,303.093364,-41.610170,0.002657,0.002944,8.117845,-1,PSZ2 G359.07-32.12,17666.435801


In [14]:
df_m2 = df_m.merge(df_specs,                                                      
               how='left',                                              
               left_on=['NAME', 'INDEX'],                                          
               right_on=['FIELD', 'INDEX'],                                           
               suffixes=('_data', '_detc'))

In [15]:
df_m2

,NAME,INDEX_PSZ2,RA_data,DEC_data,REDSHIFT,XRT_exp,N_H,INDEX,RA_VTP,RA_ERR_VTP,...,CHI2,FLUX,CNT_RATE_NET,CNT_RATE_MODEL,ECF,LUM,FLUX_corr,LUM_corr,CORR,FIELD
0,PSZ1 G023.53-36.53,NaN,313.528542,-22.841100,0.2840,NaN,NaN,1,313.521441,0.001330,...,0.9225,5.471100e-13,0.023950,0.025276,2.164500e-11,1.243400e+44,7.085648e-12,1.610333e+45,11.951048,PSZ1 G023.53-36.53
1,PSZ1 G031.41+28.75,NaN,256.263153,11.460448,0.2300,NaN,NaN,1,256.569526,0.000451,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PSZ1 G031.41+28.75,NaN,256.263153,11.460448,0.2300,NaN,NaN,2,256.568667,0.000225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PSZ1 G031.41+28.75,NaN,256.263153,11.460448,0.2300,NaN,NaN,3,256.555757,0.000139,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PSZ1 G031.41+28.75,NaN,256.263153,11.460448,0.2300,NaN,NaN,4,256.536503,0.000359,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,PSZ2 G358.98-67.26,1650.0,348.926557,-37.779104,0.1786,NaN,NaN,1,348.934928,0.002054,...,1.5510,1.588700e-12,0.081760,0.076073,2.088400e-11,1.316400e+44,2.696686e-11,2.234479e+45,15.974167,PSZ2 G358.98-67.26
864,PSZ2 G359.07-32.12,1651.0,303.018900,-41.486882,0.1496,NaN,NaN,1,303.146133,0.000335,...,0.9470,3.337700e-14,0.001527,0.001605,2.079200e-11,1.892100e+42,3.921970e-13,2.223315e+43,10.750517,PSZ2 G359.07-32.12
865,PSZ2 G359.07-32.12,1651.0,303.018900,-41.486882,0.1496,NaN,NaN,2,303.111014,0.000702,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
866,PSZ2 G359.07-32.12,1651.0,303.018900,-41.486882,0.1496,NaN,NaN,3,303.093206,0.000322,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_close_ext = df_m2[(df_m2.PSZ_dist < 6) & (df_m2.Extended == 1)]

In [17]:
df_close_ext[['NAME', 'RA_detc', 'DEC_detc', 'REDSHIFT', 'INDEX', 
              'PSZ_dist', 'EXPTIME', 'N_H', 'SN', 'FLUX_corr', 'CNT_RATE_NET', 'LUM_corr']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 467 entries, 0 to 867
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   NAME          467 non-null    object 
 1   RA_detc       467 non-null    float64
 2   DEC_detc      467 non-null    float64
 3   REDSHIFT      305 non-null    float64
 4   INDEX         467 non-null    int64  
 5   PSZ_dist      467 non-null    float64
 6   EXPTIME       467 non-null    float32
 7   N_H           0 non-null      float64
 8   SN            460 non-null    float64
 9   FLUX_corr     460 non-null    float64
 10  CNT_RATE_NET  460 non-null    float64
 11  LUM_corr      460 non-null    float64
dtypes: float32(1), float64(9), int64(1), object(1)
memory usage: 45.6+ KB


In [18]:
df_close_ext[df_close_ext.FLUX_corr.isnull()]

,NAME,INDEX_PSZ2,RA_data,DEC_data,REDSHIFT,XRT_exp,N_H,INDEX,RA_VTP,RA_ERR_VTP,...,CHI2,FLUX,CNT_RATE_NET,CNT_RATE_MODEL,ECF,LUM,FLUX_corr,LUM_corr,CORR,FIELD
157,PSZ2 G039.34+73.28,141.0,211.693221,27.729940,0.566447,NaN,NaN,16,211.732568,0.000167,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,PSZ2 G039.34+73.28,141.0,211.693221,27.729940,0.566447,NaN,NaN,19,211.703088,0.000199,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162,PSZ2 G039.34+73.28,141.0,211.693221,27.729940,0.566447,NaN,NaN,21,211.657259,0.000217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
185,PSZ2 G057.09+11.19,233.0,283.519410,26.482998,NaN,NaN,NaN,1,283.541280,0.000715,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
696,PSZ2 G283.91+73.87,1320.0,187.554916,11.796370,0.085200,NaN,NaN,2,187.597112,0.000795,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
697,PSZ2 G283.91+73.87,1320.0,187.554916,11.796370,0.085200,NaN,NaN,3,187.545628,0.001144,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
698,PSZ2 G283.91+73.87,1320.0,187.554916,11.796370,0.085200,NaN,NaN,4,187.476988,0.000883,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df_close_ext.columns

Index(['NAME', 'INDEX_PSZ2', 'RA_data', 'DEC_data', 'REDSHIFT', 'XRT_exp',
       'N_H', 'INDEX', 'RA_VTP', 'RA_ERR_VTP', 'DEC_VTP', 'DEC_ERR_VTP',
       'X_VTP', 'Y_VTP', 'X_ERR_VTP', 'Y_ERR_VTP', 'SRC_AREA', 'NET_COUNTS',
       'NET_COUNTS_ERR', 'BKG_COUNTS', 'BKG_COUNTS_ERR', 'NET_RATE',
       'NET_RATE_ERR', 'BKG_RATE', 'BKG_RATE_ERR', 'EXPTIME', 'SRC_CUTOFF',
       'FSP', 'EDGE_OF_FIELD', 'SHAPE', 'ROTANG', 'COMPONENT', 'HIGH', 'X',
       'Y', 'X_ERR', 'Y_ERR', 'RA_detc', 'DEC_detc', 'RA_ERR', 'DEC_ERR',
       'PSZ_dist', 'Extended', 'Field', 'Field_EXP', 'SN', 'CHI2', 'FLUX',
       'CNT_RATE_NET', 'CNT_RATE_MODEL', 'ECF', 'LUM', 'FLUX_corr', 'LUM_corr',
       'CORR', 'FIELD'],
      dtype='object')

In [22]:
cols = ['NAME', 'RA_detc', 'DEC_detc', 'REDSHIFT', 'INDEX', 'PSZ_dist', 'EXPTIME',
       'SN', 'CHI2', 'FLUX', 'CNT_RATE_NET', 'CNT_RATE_MODEL', 'ECF', 'LUM', 'FLUX_corr', 'LUM_corr',
       'CORR']

In [24]:
t1 = Table.from_pandas(df_close_ext[cols])
t1.write('results_table_full.fits', overwrite=True)

In [25]:
t1

NAME,RA_detc,DEC_detc,REDSHIFT,INDEX,PSZ_dist,EXPTIME,SN,CHI2,FLUX,CNT_RATE_NET,CNT_RATE_MODEL,ECF,LUM,FLUX_corr,LUM_corr,CORR
str18,float64,float64,float64,int64,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
PSZ1 G023.53-36.53,313.5192459279641,-22.811192599451434,0.2840000092983246,1,1.8666280661228891,2716.9482,8.5801,0.9225,5.4711e-13,0.02395,0.0252761,2.1645e-11,1.2434e+44,7.085647644377957e-12,1.6103332567526735e+45,11.951047585271622
PSZ1 G031.41+28.75,256.3524228060791,11.435045572929392,0.23,22,5.466414508179784,216357.27,-1.0,2945.86,1.706e-15,5.248e-05,8.55077e-05,1.9951e-11,2.4421e+41,3.737449486572262e-15,5.350073500092686e+41,1.1907675771232487
PSZ1 G031.41+28.75,256.2842552229166,11.378437505810556,0.23,33,5.074730659124593,197958.08,5.2303,0.492,3.9996e-15,0.0001508,0.000177234,2.2567e-11,5.7253e+41,1.492424613399746e-14,2.1363582956039523e+42,2.731434676967062
PSZ1 G055.72+17.58,276.3803066233068,27.741361425895942,0.1940000057220459,1,4.293797561529516,3424.389,15.1945,1.122,1.2437e-12,0.05376,0.0598634,2.0776e-11,1.2305e+44,1.7919199512193083e-11,1.772901423152978e+45,13.407975807825908
PSZ1 G057.42-10.77,304.36687269578346,16.062365346049575,0.136,1,1.9135608557965353,2917.1406,15.5937,0.686,1.6882e-12,0.06325,0.0802949,2.1025e-11,7.8154e+43,3.354087165369074e-11,1.5527504343220863e+45,18.86783062059634
PSZ1 G083.35+76.41,203.70014715336345,37.18210039979622,--,1,0.9253717082144188,46343.074,-1.0,6.0216,2.9848e-12,0.1743,0.126522,2.3591e-11,7.229e+43,2.2028305527274626e-10,5.3351186229116954e+45,72.80161326479036
PSZ1 G083.35+76.41,203.6204264527387,37.209131244254884,--,2,3.3910404629742787,52051.797,2.8819,0.3393,8.2954e-14,0.006879,0.00362458,2.2887e-11,2.0091e+42,2.6620946939348127e-13,6.447446114213217e+42,2.209121554035746
PSZ1 G103.50+31.36,264.23998368261863,72.56058825187006,--,1,0.982848402974421,2846.7449,-1.0,2.09,1.2301e-13,0.00905,0.0057721,2.1311e-11,2.9792e+42,4.742321789246655e-13,1.1485509368769723e+43,2.855232736563414
PSZ1 G103.94+25.81,283.03587113474066,72.98968469360932,0.0989999994635582,1,1.7836576822735373,2893.2185,8.7933,0.7554,5.7691e-13,0.02708,0.0272064,2.1205e-11,1.3677e+43,5.0905144357552035e-12,1.2068254309653832e+44,7.823758360498523


In [ ]:
mask = df_close_ext[['NAME', 'RA_detc']].duplicated('NAME', keep='first')

In [ ]:
df_close_ext2 = df_close_ext.copy()

In [ ]:
df_close_ext2.NAME[mask] = ''

In [ ]:
df_close_ext2[['NAME', 'RA_detc', 'DEC_detc', 'REDSHIFT', 'INDEX', 'PSZ_dist', 'EXPTIME']][:25]

In [ ]:
t2 = Table.from_pandas(df_close_ext2[['NAME', 'RA_detc', 'DEC_detc', 'REDSHIFT', 'INDEX', 'PSZ_dist', 'EXPTIME']])

In [ ]:
t2.write('table_test.tex', format='latex')

In [ ]:
df_close_ext2[['NAME', 'RA_detc', 'DEC_detc', 'REDSHIFT', 'INDEX', 'PSZ_dist', 'EXPTIME']][:50].to_latex('table_test.tex', index=False, float_format='%0.2f') 

In [ ]:
t2.write('results_table.fits')

In [ ]:
t2['REDSHIFT'].filled